# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
!pip install --upgrade gdown

# Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

!unzip -q libriphone.zip
!ls libriphone

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1
To: /kaggle/working/libriphone.zip
100%|█████████████████████████████████████████| 384M/384M [00:01<00:00, 235MB/s]
feat  test_split.txt  train_labels.txt	train_split.txt


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [1]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

/home/frankchiang/local/anaconda3/envs/ml2023_hw2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [2]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Hyper-parameters

In [4]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 33#11   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75   # the ratio of data used for training, the rest will be used for validation

# training parameters
use_model = ""#"LSTM"
seed = 16588412          # random seed
batch_size = 2048#256        # batch size
num_epoch = 500         # the number of training epoch
early_break_num = 5    # early break epoch count
learning_rate = 1e-4      # learning rate
model_path = './model.ckpt'  # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
hidden_layers = 8          # the number of hidden layers
hidden_dim = 256           # the hidden dim
sequence_length = 4
embedding_dim = 10

# Model
Feel free to modify the structure of the model.

In [5]:
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.bn=nn.BatchNorm1d(input_dim)
        self.dropout=nn.Dropout(0.25)

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        #print("x.shape =", x.shape)
        # print (x)
        # x_int = torch.mul(x, 10000).long()
        # print(x_int.shape)
        # print (x_int)
        # x_emb = self.embedding(x_int)
        # print(x_emb.shape)
        #print (x_emb)
        x_bn = self.bn(x)
        x_dr = self.dropout(x_bn)

        x_out = self.block(x_dr)
        return x_out


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x
    
class RNN_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_layers, output_dim=41, embedding_dim=10):
        super(RNN_LSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_layers = hidden_layers
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.seq_length = sequence_length

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.bn=nn.BatchNorm1d(input_dim)
        self.dropout=nn.Dropout(0.25)
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                            num_layers=hidden_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        #x_ = self.embedding(x)
        x_ = self.bn(x)
        x_ = self.dropout(x_)
        x_, (h_n, c_n) = self.lstm(x_)
        #print (x.shape)
        #print (x_.shape)
        #x_ = (x_[:, -1, :])
        x_ = self.fc(x_)
        return x_

DEVICE: cpu


# Dataloader

In [6]:
from torch.utils.data import DataLoader
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:08, 286.14it/s]


[INFO] train set
torch.Size([1577444, 1287])
torch.Size([1577444])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:03, 272.10it/s]

[INFO] val set
torch.Size([539350, 1287])
torch.Size([539350])


# Training

In [7]:
# create model, define a loss function, and optimizer
#model = RNN_LSTM(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layers).to(device)
if use_model == "LSTM":
    model = RNN_LSTM(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layers).to(device)
else:
    model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08,amsgrad=False, weight_decay=1e-4)

best_acc = 0.0
not_improve_count = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        #print (outputs.shape)
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    #if epoch<50 or (epoch+1)%200==0:
    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        not_improve_count = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        #if epoch<50 or (epoch+1)%200==0:
        print(f'saving model with acc {best_acc/len(val_set):.5f}')
    else:
        not_improve_count += 1

    # early break
    if not_improve_count > early_break_num:
        print(f'Early break. Best acc = {best_acc/len(val_set):3.5f}')
        break


100%|██████████| 264/264 [00:06<00:00, 38.14it/s]


[001/500] Train Acc: 0.35637 Loss: 2.32578 | Val Acc: 0.48933 loss: 1.73206
saving model with acc 0.48933


100%|██████████| 264/264 [00:06<00:00, 38.29it/s]


[002/500] Train Acc: 0.47210 Loss: 1.79137 | Val Acc: 0.53681 loss: 1.53609
saving model with acc 0.53681


100%|██████████| 264/264 [00:06<00:00, 38.61it/s]


...

[165/500] Train Acc: 0.64718 Loss: 1.15038 | Val Acc: 0.70671 loss: 0.93844


100%|██████████| 264/264 [00:07<00:00, 34.05it/s]


[166/500] Train Acc: 0.64697 Loss: 1.15004 | Val Acc: 0.70654 loss: 0.93960


100%|██████████| 264/264 [00:07<00:00, 34.02it/s]

[167/500] Train Acc: 0.64766 Loss: 1.14987 | Val Acc: 0.70695 loss: 0.93884
Early break. Best acc = 0.70701


In [8]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

22

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [9]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:03, 264.39it/s]

[INFO] test set
torch.Size([527364, 1287])


In [10]:
# load model
if use_model == "LSTM":
    model = RNN_LSTM(input_dim=input_dim, hidden_dim=hidden_dim, hidden_layers=hidden_layers).to(device)
else:
    model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [11]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 258/258 [00:05<00:00, 45.32it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [12]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))